## Load Necessary Libraries

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

## Read Data

### Load Data

In [66]:
df = pd.read_stata('lendingclub_train.dta')
# df.head()
# df.describe()
columns_to_drop = [
    "index",
    "depvar",
    "total_acc", 
    "out_prncp", 
    "out_prncp_inv", 
    "total_pymnt", 
    "total_pymnt_inv", 
    "total_rec_prncp", 
    "total_rec_int", 
    "total_rec_late_fee", 
    "recoveries", 
    "collection_recovery_fee",
    "last_pymnt_amnt", 
    # "last_fico_range_high", 
    # "last_fico_range_low", 
    # "tot_coll_amt", 
    # "tot_cur_bal", 
    "initial_list_status1", 
    "initial_list_status2", 
    "elapsed_t",
    "purpose1",
    "addr_state1",
    "elapsed_t",
    "debt_settlement_flag1",
    "term1",
    "mths_since_last_delinq1",
    "mths_since_last_major_derog1",
    "mths_since_last_record1",
    "mths_since_rcnt_il1",
    "mths_since_recent_bc1",
    "mths_since_recent_bc_dlq1",
    "mths_since_recent_inq1",
    "mths_since_recent_revol_delinq1"
]

issue_d_count = 1
while(issue_d_count <= 118):
    word_tmp = "issue_d" + str(issue_d_count)
    columns_to_drop.append(word_tmp)
    issue_d_count += 1


### Undersampling
필요없으면 건너뛰어도 무방

In [67]:
# Separate majority and minority classes
df_majority = df[df['depvar'] == 0]
df_minority = df[df['depvar'] == 1]

# Count number of instances in the minority class
minority_count = len(df_minority)

# Undersample the majority class
df_majority_undersampled = df_majority.sample(n=minority_count, random_state=42)

# Combine minority class with the undersampled majority class
df_undersampled = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

# Now split into features and target, and then into training and testing sets
y = df_undersampled['depvar']
X_original = df_undersampled
X = df_undersampled.drop(columns=columns_to_drop)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Drop Unnecessary Columns

In [16]:
# y = df['depvar']
# X = df.drop(columns=columns_to_drop)
# X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874335 entries, 0 to 874334
Columns: 188 entries, loan_amnt to mths_since_recent_revol_delinq11
dtypes: float64(6), int16(5), int32(6), int8(171)
memory usage: 211.0 MB


### Hyperparameter Tuning

In [17]:
# Assuming X and y are your features and target variable
# X = your_data_features
# y = your_data_target

# Splitting the data into training, validation and testing sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [5, 10]  ###############################<==이부분을 각자 하나씩 맡으면 됩니다!!
}

# Create a Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Evaluate on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_report = classification_report(y_val, y_val_pred)

print("Validation Accuracy: ", val_accuracy)
print("Validation Classification Report: \n", val_report)


## Model Training

In [33]:

# Splitting the dataset into training and testing sets
X_train_model, X_test, y_train_model, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Creating a Random Forest Classifier
# rf = RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], min_samples_split=best_params['min_samples_split'], random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=30, min_samples_split=10, random_state=42)

# Training the model
rf.fit(X_train_model, y_train_model)


RandomForestClassifier(max_depth=30, min_samples_split=10, n_estimators=200,
                       random_state=42)

## Evaluation

In [34]:
# Predicting the Test set results
y_pred = rf.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(accuracy)
print(report)


0.867250911657452
              precision    recall  f1-score   support

         0.0       0.90      0.83      0.86     16966
         1.0       0.84      0.91      0.87     17038

    accuracy                           0.87     34004
   macro avg       0.87      0.87      0.87     34004
weighted avg       0.87      0.87      0.87     34004



In [12]:
y_pred_proba = rf.predict_proba(X_test)
y_pred_proba

array([[0.9794646 , 0.0205354 ],
       [0.72461854, 0.27538146],
       [0.97009209, 0.02990791],
       ...,
       [0.94075408, 0.05924592],
       [0.95923372, 0.04076628],
       [0.91520273, 0.08479727]])

## Save Model

In [35]:
import joblib

# save
joblib.dump(rf, "random_forest_final.joblib")

['random_forest_final.joblib']

## Setting Threshold

In [48]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [49]:
y_val_pred = rf.predict_proba(X_val)
y_val_pred1 = y_val_pred[:,1]

In [50]:
auc_score = roc_auc_score(y_val,y_val_pred1)
auc_score

0.9233447763258751

In [78]:
# Gain domain(금액 모두 돌려받고 추가로 이자소득이 나옴)
# = TN*펀딩액*이자율 + TP*펀딩액*미국국채이자율
# Loss domain(이자소득을 따지기 이전에 원금이 안돌아옴)
# = FN * (펀딩금액 * (1+국채이자율) - 부도전까지갚은금액 - 부도전까지갚은이자) + FP*펀딩액*(랜딩클럽이자율 - 미국국채이자율)

thresholds = np.arange(0.0, 1.01, 0.01)

metrics_df = pd.DataFrame(columns=['threshold', 'gain_or_loss'])

for threshold in thresholds:
    
    gov_bond_rate = 0.04

    X_original_train, X_original_temp, y_train, y_temp = train_test_split(X_original, y, test_size=0.4, random_state=42)
    X_original_val, X_original_test, y_val, y_test = train_test_split(X_original_temp, y_temp, test_size=0.5, random_state=42)


    X_val_thres = X_original_val
    X_val_thres['tn_gain'] = X_val_thres['funded_amnt'] * X_val_thres['int_rate'] * 36
    X_val_thres['tp_gain'] = X_val_thres['funded_amnt'] * gov_bond_rate
    X_val_thres['fn_loss'] = X_val_thres['funded_amnt'] * (1+gov_bond_rate) - X_val_thres['total_rec_prncp'] 
    X_val_thres['fp_loss'] = X_val_thres['funded_amnt'] * (X_val_thres['int_rate'] - gov_bond_rate)

    X_val_thres['rf_pred_proba'] = rf.predict_proba(X_val)[:,1]
    # X_val_thres['rf_pred_proba']
    X_val_thres['rf_pred_onehot'] = X_val_thres['rf_pred_proba'].apply(lambda row: 1 if row > threshold else 0)

    X_val_thres['predict_result'] = X_val_thres.apply(lambda row: 'tp' if ((row['depvar'] == row['rf_pred_onehot']) and (row['rf_pred_onehot'] == 1))
                                            else 'tn' if ((row['depvar'] == row['rf_pred_onehot']) and (row['rf_pred_onehot'] == 0))
                                            else 'fp' if (row['rf_pred_onehot'] == 1) 
                                            else 'fn', axis=1)

    X_val_thres['gain_or_loss'] = X_val_thres.apply(lambda row: row['tn_gain'] if (row['predict_result'] == 'tn')
                                                    else row['tp_gain'] if (row['predict_result'] == 'tp')
                                                    else row['fn_loss']*-1 if (row['predict_result'] == 'fn')
                                                    else row['fp_loss']*-1, axis=1)

    X_val_thres['gain_or_loss'].sum()

    metrics_df.loc[len(metrics_df.index)] = [threshold, X_val_thres['gain_or_loss'].sum()]


display(metrics_df)

/var/folders/lx/3vyc9qlj1cqg5xytpq0yk4680000gn/T/ipykernel_1464/1382684220.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_val_thres['tn_gain'] = X_val_thres['funded_amnt'] * X_val_thres['int_rate'] * 36
/var/folders/lx/3vyc9qlj1cqg5xytpq0yk4680000gn/T/ipykernel_1464/1382684220.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_val_thres['tp_gain'] = X_val_thres['funded_amnt'] * gov_bond_rate
/var/folders/lx/3vyc9qlj1cqg5xytpq0yk4680000gn/T/ipykernel_1464/1382684220.py:16: PerformanceWarning: DataFrame is highly fragm

,threshold,gain_or_loss
0,0.00,-1.616925e+07
1,0.01,-1.383347e+07
2,0.02,8.070301e+06
3,0.03,6.280554e+07
4,0.04,1.466678e+08
...,...,...
96,0.96,1.447555e+09
97,0.97,1.447139e+09
98,0.98,1.446936e+09
99,0.99,1.447171e+09


In [79]:
optimal_threshold = metrics_df.threshold[metrics_df.gain_or_loss.idxmax()]
optimal_threshold

0.8300000000000001